In [7]:
'''This script is for generating data
1. Provide desired path to store images.
2. Press 'c' to capture image and display it.
3. Press any button to continue.
4. Press 'q' to quit.
'''

import cv2

camera = cv2.VideoCapture(0)
ret, img = camera.read()



count = 0
while True:
    name = path + str(count)+".jpg"
    ret, img = camera.read()
    cv2.imshow("img", img)


    if cv2.waitKey(20) & 0xFF == ord('c'):
        cv2.imwrite(name, img)
        cv2.imshow("img", img)
        count += 1
        if cv2.waitKey(0) & 0xFF == ord('q'):

            break;

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [8]:
!pip uninstall opencv-python opencv-contrib-python
!pip install opencv-python opencv-contrib-python


^C
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Farah Elsoussi\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
!pip install opencv-contrib-python==4.6.0.66 --user

  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)


In [6]:
!pip show opencv-contrib-python

Name: opencv-contrib-python
Version: 4.6.0.66
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/skvark/opencv-python
Author: 
Author-email: 
License: MIT
Location: c:\users\farah elsoussi\appdata\roaming\python\python310\site-packages
Requires: numpy, numpy, numpy, numpy
Required-by: 


In [17]:
ls 0


 Volume in drive C is OS
 Volume Serial Number is E86A-A53B

 Directory of C:\Users\Farah Elsoussi\Desktop\basics of mobile robotics\project\0

11/17/2024  03:27 AM    <DIR>          .
11/17/2024  03:44 AM    <DIR>          ..
11/17/2024  03:27 AM            91,956 0.jpg
11/17/2024  03:27 AM            93,952 1.jpg
11/17/2024  03:27 AM            78,668 10.jpg
11/17/2024  03:27 AM            73,794 11.jpg
11/17/2024  03:27 AM           104,480 12.jpg
11/17/2024  03:27 AM           103,087 13.jpg
11/17/2024  03:27 AM            97,971 14.jpg
11/17/2024  03:27 AM            78,156 15.jpg
11/17/2024  03:27 AM            84,619 16.jpg
11/17/2024  03:27 AM            83,920 17.jpg
11/17/2024  03:27 AM            86,463 18.jpg
11/17/2024  03:27 AM            84,851 19.jpg
11/17/2024  03:27 AM            91,872 2.jpg
11/17/2024  03:27 AM            73,600 20.jpg
11/17/2024  03:27 AM            97,544 21.jpg
11/17/2024  03:27 AM           100,200 22.jpg
11/17/2024  03:27 AM            94,912 2

In [35]:
import cv2
from cv2 import aruco
import numpy as np
import yaml
from pathlib import Path
from tqdm import tqdm

# Calibration images directory (update this path to your dataset)
calib_imgs_path = Path("0")  # Update this path

# ArUco dictionary and board parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_1000)  # Change dictionary if needed
marker_length = 3.75  # Marker side length in centimeters (or any unit)
marker_separation = 0.5  # Separation between markers in centimeters (or any unit)
board = aruco.GridBoard_create(4, 5, marker_length, marker_separation, aruco_dict)
aruco_params = aruco.DetectorParameters_create()

# Check if calibration images exist
if not calib_imgs_path.exists():
    raise FileNotFoundError(f"Calibration images directory {calib_imgs_path} does not exist.")

# Get list of calibration images
image_files = sorted(calib_imgs_path.glob("*.jpg"))
if not image_files:
    raise FileNotFoundError(f"No .jpg images found in directory {calib_imgs_path}.")

# Variables for storing detected corners and IDs
all_corners = []
all_ids = []
image_size = None

print("Processing images...")
print(image_files)
for image_path in tqdm(image_files):
    
    # Read each image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Warning: Unable to read {image_path.name}")
        continue

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    # If markers are detected
    if ids is not None and len(ids) > 0:
        all_corners.append(corners)
        all_ids.append(ids)
        if image_size is None:
            image_size = gray.shape[::-1]  # (width, height)

print("Calibrating camera...")
# Perform camera calibration
if len(all_corners) > 0:
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = aruco.calibrateCameraAruco(
        all_corners, all_ids, np.array([len(ids) for ids in all_ids]),
        board, image_size, None, None
    )

    if ret:
        print(f"Calibration successful with RMS error: {ret}")
        print(f"Camera matrix:\n{camera_matrix}")
        print(f"Distortion coefficients:\n{dist_coeffs}")

        # Save calibration parameters to a YAML file
        calibration_data = {
            "camera_matrix": camera_matrix.tolist(),
            "distortion_coefficients": dist_coeffs.tolist(),
        }
        with open("aruco_calibration.yaml", "w") as yaml_file:
            yaml.dump(calibration_data, yaml_file, default_flow_style=False)
        print("Calibration parameters saved to aruco_calibration.yaml.")
    else:
        print("Calibration failed.")
else:
    print("No valid markers detected in the images. Calibration failed.")


Processing images...
[WindowsPath('0/0.jpg'), WindowsPath('0/1.jpg'), WindowsPath('0/10.jpg'), WindowsPath('0/11.jpg'), WindowsPath('0/12.jpg'), WindowsPath('0/13.jpg'), WindowsPath('0/14.jpg'), WindowsPath('0/15.jpg'), WindowsPath('0/16.jpg'), WindowsPath('0/17.jpg'), WindowsPath('0/18.jpg'), WindowsPath('0/19.jpg'), WindowsPath('0/2.jpg'), WindowsPath('0/20.jpg'), WindowsPath('0/21.jpg'), WindowsPath('0/22.jpg'), WindowsPath('0/23.jpg'), WindowsPath('0/24.jpg'), WindowsPath('0/25.jpg'), WindowsPath('0/26.jpg'), WindowsPath('0/27.jpg'), WindowsPath('0/28.jpg'), WindowsPath('0/29.jpg'), WindowsPath('0/3.jpg'), WindowsPath('0/30.jpg'), WindowsPath('0/31.jpg'), WindowsPath('0/32.jpg'), WindowsPath('0/33.jpg'), WindowsPath('0/34.jpg'), WindowsPath('0/35.jpg'), WindowsPath('0/36.jpg'), WindowsPath('0/37.jpg'), WindowsPath('0/38.jpg'), WindowsPath('0/39.jpg'), WindowsPath('0/4.jpg'), WindowsPath('0/40.jpg'), WindowsPath('0/41.jpg'), WindowsPath('0/42.jpg'), WindowsPath('0/43.jpg'), WindowsP

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 96.13it/s]

Calibrating camera...
No valid markers detected in the images. Calibration failed.


In [47]:
"""
This code assumes that images used for calibration are of the same arUco marker board provided with code

"""

import cv2
from cv2 import aruco
import yaml
import numpy as np
from pathlib import Path
from tqdm import tqdm

# root directory of repo for relative path specification.
root = Path("")

# Set this flsg True for calibrating camera and False for validating results real time
calibrate_camera = False

# Set path to the images
calib_imgs_path = root.joinpath("project_pics")

# For validating results, show aruco board to camera.
aruco_dict = aruco.getPredefinedDictionary( aruco.DICT_6X6_1000 )

#Provide length of the marker's side
markerLength = 3.75  # Here, measurement unit is centimetre.

# Provide separation between markers
markerSeparation = 0.5   # Here, measurement unit is centimetre.

# create arUco board
board = aruco.GridBoard_create(4, 5, markerLength, markerSeparation, aruco_dict)

'''uncomment following block to draw and show the board'''
#img = board.draw((864,1080))
#cv2.imshow("aruco", img)

arucoParams = aruco.DetectorParameters_create()

if calibrate_camera == True:
    img_list = []
    calib_fnms = calib_imgs_path.glob('*.jpg')
    print('Using ...', end='')
    for idx, fn in enumerate(calib_fnms):
        print(idx, '', end='')
        img = cv2.imread( str(root.joinpath(fn) ))
        img_list.append( img )
        h, w, c = img.shape
    print('Calibration images')

    counter, corners_list, id_list = [], [], []
    first = True
    for im in tqdm(img_list):
        img_gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
        
        corners, ids, rejectedImgPoints = aruco.detectMarkers(img_gray, aruco_dict, parameters=arucoParams)
        if first == True:
            corners_list = corners
            id_list = ids
            first = False
        else:
            corners_list = np.vstack((corners_list, corners))
            id_list = np.vstack((id_list,ids))
        counter.append(len(ids))
    print('Found {} unique markers'.format(np.unique(ids)))

    counter = np.array(counter)
    print ("Calibrating camera .... Please wait...")
    #mat = np.zeros((3,3), float)
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(corners_list, id_list, counter, board, img_gray.shape, None, None )

    print("Camera matrix is \n", mtx, "\n And is stored in calibration.yaml file along with distortion coefficients : \n", dist)
    data = {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}
    with open("calibration.yaml", "w") as f:
        yaml.dump(data, f)

else:
    camera = cv2.VideoCapture(0)
    ret, img = camera.read()

    with open('calibration.yaml') as f:
        print(f)
        loadeddict = yaml.load(f,Loader=yaml.FullLoader)
    mtx = loadeddict.get('camera_matrix')
    dist = loadeddict.get('dist_coeff')
    mtx = np.array(mtx)
    dist = np.array(dist)

    ret, img = camera.read()
    img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    h,  w = img_gray.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    pose_r, pose_t = [], []
    while True:
        ret, img = camera.read()
        img_aruco = img
        im_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        h,  w = im_gray.shape[:2]
        dst = cv2.undistort(im_gray, mtx, dist, None, newcameramtx)
        corners, ids, rejectedImgPoints = aruco.detectMarkers(dst, aruco_dict, parameters=arucoParams)
        #cv2.imshow("original", img_gray)
        if corners == None:
            print ("pass")
        else:

            ret, rvec, tvec = aruco.estimatePoseBoard(corners, ids, board, newcameramtx, dist) # For a board
            print ("Rotation ", rvec, "Translation", tvec)
            if ret != 0:
                img_aruco = aruco.drawDetectedMarkers(img, corners, ids, (0,255,0))
                img_aruco = aruco.drawAxis(img_aruco, newcameramtx, dist, rvec, tvec, 10)    # axis length 100 can be changed according to your requirement

            if cv2.waitKey(0) & 0xFF == ord('q'):
                break;
        cv2.imshow("World co-ordinate frame axes", img_aruco)

cv2.destroyAllWindows()

<_io.TextIOWrapper name='calibration.yaml' mode='r' encoding='cp1252'>
Rotation  [[0. 0. 0.]] Translation [[0. 0. 0.]]


AttributeError: module 'cv2.aruco' has no attribute 'drawAxis'

In [54]:
import cv2
from cv2 import aruco
import numpy as np
import yaml

# Load calibration data
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Load the ArUco dictionary and board
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_1000)
arucoParams = aruco.DetectorParameters_create()

# Set up the GridBoard parameters
markerLength = 3.75  # cm
markerSeparation = 0.5  # cm
board = aruco.GridBoard_create(4, 5, markerLength, markerSeparation, aruco_dict)

# Initialize camera
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("Error: Could not open the camera.")
    exit()

# Get the optimal camera matrix
ret, img = camera.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()

h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Processing loop
print("Press 'q' to exit.")
while True:
    ret, img = camera.read()
    if not ret:
        print("Error: Could not read from the camera.")
        break

    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Undistort the image
    dst = cv2.undistort(img_gray, mtx, dist, None, newcameramtx)
    
    # Detect markers
    corners, ids, rejectedImgPoints = aruco.detectMarkers(dst, aruco_dict, parameters=arucoParams)
    
    if ids is not None:  # Markers detected
        # Estimate pose of the board
        rvec = np.zeros((1, 3), dtype=np.float32)
        tvec = np.zeros((1, 3), dtype=np.float32)
        ret = aruco.estimatePoseBoard(corners, ids, board, newcameramtx, dist, rvec, tvec)
        
        if ret:
            print("Rotation Vector (rvec):", rvec)
            print("Translation Vector (tvec):", tvec)
            
            # Draw markers and axis
            img_aruco = aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 0))
            #img_aruco = cv2.drawFrameAxis(img_aruco, newcameramtx, dist, rvec, tvec, 10)  # Axis length = 10
            img_aruco = cv2.drawFrameAxes(img_aruco, newcameramtx, dist, rvec, tvec, 20)

            # Display the output
            cv2.imshow("World Coordinate Frame Axes", img_aruco)
        else:
            print("Pose estimation failed.")
    else:
        print("No markers detected.")

    # Quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
camera.release()
cv2.destroyAllWindows()


Press 'q' to exit.
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (tvec): [[0. 0. 0.]]
Rotation Vector (rvec): [[0. 0. 0.]]
Translation Vector (